# Workshop 2: Heat equation

We now turn to solving a classical (linear) partial differential equation (PDE), the [heat equation](https://en.wikipedia.org/wiki/Heat_equation)

$$
\alpha \Delta u(X,t) = \frac{\partial u(X,t)}{\partial t} ,
$$

using FEM. The solution's time integration will be discretized with a first order [backward differentiation](https://en.wikipedia.org/wiki/Backward_differentiation_formula) approach, otherwise known as [Backward Euler](https://en.wikipedia.org/wiki/Backward_Euler_method).

## FEM discretization

Start by discretizing (by hand) the heat equation in the space variables only, using FEM. Treat the time derivative of heat as a forcing function for now. 

The final result should be a matrix system of equations in the form $\alpha L u = M \frac{\partial u}{\partial t}$.

## Time discretization

Discretize the heat's time derivative, which we previously considered to be a forcing function, using backward finite differences and a time step $\Delta t$. Collect the unknowns on the left-hand side of the discretized matrix equation. 

The final result should be $(M - \Delta t \alpha  L) u^{t+1} = M u^t$.

## Solve heat equation

1. Use `pbatoolkit`'s `pbatoolkit.math.linalg.ldlt` function to factorize the discretized heat equation's lead matrix for a given mesh of your choosing. 
2. Our discrete heat equation is now an initial value problem [IVP](https://en.wikipedia.org/wiki/Initial_value_problem#:~:text=In%20multivariable%20calculus%2C%20an%20initial,given%20point%20in%20the%20domain.), such that the initial condition $u(t=0) = u^0$ must be given. Notice that $u^t$ must be discretized at FEM nodes, since it is to be projected (in the Galerkin sense) via the mass matrix $M$. Thus, pick a subset of FEM nodes, and initialize their heat values to $1$ (or something else). Set the initial heat values of other nodes to $0$ (or something else). You can use the function `pbatoolkit.geometry.aabb` to aid in selecting FEM nodes in box regions, or select them manually in `polyscope`, [Blender](https://www.blender.org/), [MeshLab](https://www.meshlab.net/) or other. Otherwise, pick at random.
3. Solve the IVP for a certain number of time steps $T$ and visualize each time step's heat distribution over the mesh using `polyscope`.
4. What happens when you change the diffusivity constant $\alpha$, the time step size $\Delta t$?

## Linear solver

1. Choose a different linear solver for the IVP. Try, for instance, [`scipy`'s sparse LU factorization](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.splu.html). Analyze the computational cost of different approaches.
2. If the results are different, consider why.
3. What happens if a dense linear solver is used, rather than a sparse one?

## Heat sources

How should you model a heat source? These are regions in your domain (i.e. the FEM mesh) which emit constant heat.

One way is to impose Dirichlet boundary conditions, i.e. constrain the solution to satisfy $u(X_D,t) = d(X_D,t)$ for some known function $d(X,t)$, where $X_D$ are positions that are part of the heat source. 

1. On our discrete heat equation, pick vertices that are part of the heat source, and set their corresponding entries in the solution coefficient vector $u$ to your preferred heat source's temperature. You can use `pbatoolkit.geometry.aabb` to pick all vertices inside the prescribed axis-aligned box, to facilitate vertex selection.
2. Impose these Dirichlet boundary conditions as described in the [documentation](./README.md)'s section on boundary conditions. Once the known degrees of freedom are collected (in a list or array), you can use [numpy.setdiff1d](https://numpy.org/doc/stable/reference/generated/numpy.setdiff1d.html) to get the unknowns. Using the index lists, you can extract submatrices of the lead sparse matrix via [slicing with scipy](https://stackoverflow.com/questions/13352280/slicing-sparse-matrices-in-scipy-which-types-work-best).

## Smoothing

The physical interpretation of "heat" that we associated with the solution vector $u$ was not necessary. Can you apply the heat/diffusion equation to some other functions? 

1. Solve the heat equation on each spatial position of the FEM mesh's node positions. The initial conditions for this problem should be the initial FEM mesh's node positions. You can treat this as 3 separate IVPs for each spatial dimension. Visualize the mesh using the new positions at each time step.
2. Can the discrete heat equation's lead matrix be re-used?
3. Change parameters $\alpha, \Delta t$ and visualize results.
4. Constrain some nodal positions to be fixed. What happens?